In [2]:
import numpy as np
import pandas as pd
import re 

In [15]:
def read_qdp(file_name, make_excel = False, x = 'bias', y = 'V fb (V)'):
    '''
    Accepts a raw sting literal for the filename and path.
    x and y are strings which name the first 2 columns, default to bias
    and V fb (V)
    If make_excel is set to True then excel file is created in the name
    of the input file.
    Return the a pandas dataframe with the data. 
    '''
    #name first two cols after inputs
    cols = {0: x, 1: y}
    header = 0
    
    #precompile the regexes for use in loop
    col_nam_re = re.compile(r'la\sy(\d)\s(.+)')
    data_match_re = re.compile(r'^-?(\d.+)')
    
    with open(file_name) as file:
        for line in file:
            column_match = col_nam_re.match(line)
            if column_match:
                #remove any formatting from col names
                cols[int(column_match.group(1))] = re.sub(r'(\\d|\\u)',r' ', column_match.group(2))
            else:
                #once data is matched, loop break and record header
                data_match = data_match_re.match(line)
                if data_match:
                    break
            header += 1
    #reads table with whitespace seperators (tab or space)
    df = pd.read_csv(file_name, sep='\s+', header=None, skiprows=header, index_col=False)
    
    #make list of col names for df and number ones without labels
    col_names = []
    for i in range(len(df.columns)):
        try:
             col_names.append(cols[i])
        except:
            col_names.append(str(i))
        i += 1
            
    #set column names and sort by x vals    
    df.columns = col_names
    df.sort_values(by=[x], inplace=True)
    print(df.iloc[:,0][1])
    if df.iloc[:,0][1] * df.iloc[:,1][1] < 0:
        df.iloc[:,1] *= -1
    
    if make_excel:
        #find name for excel sheet from end of path
        name_match = re.search(r'(\\?((?:.(?!\\))+)$)', file_name)
        name = re.sub(r'(\.qdp|\\)', r'', name_match.group(1))
        #write to file and save
        writer = pd.ExcelWriter(str(name)+'.xlsx')
        df.to_excel(writer,'Sheet1',index=False)
        writer.save()
        
    return df


In [16]:
df1 = read_qdp(r'C:\Users\callu\Dropbox\University\Year 3\Block 2b\Project\IVW17C04P03ShClosed_26.91mK_BB3.20K_magnet0uA_HTRV0.00mV_HTRI33.73mA_PBB0pc.qdp')

-29.9932264042


In [17]:
print(df1)

         bias  V fb (V)             2  T bath  (mK)  T BB  (K)  V HTR  (mV)  \
1  -29.993226  0.191534  6.025430e-06      26.90379   3.204672          0.0   
3  -28.991023  0.185354           NaN      26.89577   3.204466          0.0   
5  -28.007041  0.179256           NaN      26.89978   3.204298          0.0   
7  -27.004837  0.173072           NaN      26.90379   3.204341          0.0   
9  -26.002633  0.166861           NaN      26.90780   3.204341          0.0   
11 -25.000429  0.160649           NaN      26.90379   3.204298          0.0   
13 -23.998226  0.154450  2.992630e-06      26.90580   3.204259          0.0   
15 -22.996022  0.148266           NaN      26.90580   3.204384          0.0   
17 -21.993818  0.142071  4.380210e-06      26.90580   3.204466          0.0   
19 -21.501827  0.139056  3.730520e-06      26.90780   3.204423          0.0   
21 -20.991614  0.135907  5.015180e-06      26.91583   3.204629          0.0   
23 -20.499623  0.132890           NaN      26.91784 